In [45]:
import json
import os
import pandas as pd
import string 
import pandas as pd 

from itertools import groupby
from operator import itemgetter

In [15]:
# VARIABLES
SPLIT = "test"

PATH_FC = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/vitaminc_factchecking/{}.jsonl".format(SPLIT)
PATH_REVISION = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/vitaminc_flagging/{}.jsonl".format(SPLIT)

SAMPLE = {
    "claim": "",
    "reference": "",
    "target": "",
    "score": None,
    "label": None
}

In [3]:
def load_jsonl_file(file_path, dataclass=None):
    content = []
    with open(file_path, "r", encoding="utf-8") as f:
        for entry in f.readlines():
            if dataclass:
                content.append(dacite.from_dict(data_class=dataclass, data=json.loads(entry)))
            else:
                content.append(json.loads(entry))
    return content


In [4]:
fc_data = load_jsonl_file(PATH_FC)
fc_revision = load_jsonl_file(PATH_REVISION)

In [47]:
df_fc_data = pd.DataFrame.from_dict(fc_data)

df_fc_data.head(3)

,unique_id,case_id,wiki_revision_id,label,claim,evidence,page,revision_type,FEVER_id
0,5ed4de07c9e77c000848a180_1,5ed4de07c9e77c000848a180,927477259,NOT ENOUGH INFO,Westlife made under 23.5 million sales in the ...,According to the British Phonographic Industry...,Westlife,real,NaN
1,5ed4de07c9e77c000848a180_2,5ed4de07c9e77c000848a180,927477259,REFUTES,Westlife made under 23.5 million sales in the ...,According to the British Phonographic Industry...,Westlife,real,NaN
2,5ed4de07c9e77c000848a180_3,5ed4de07c9e77c000848a180,927477259,NOT ENOUGH INFO,Westlife sold more than 1 million video albums...,According to the British Phonographic Industry...,Westlife,real,NaN


### Create examples with score==0

In [6]:
# sort by multiple evidence 
fc_data_sorted = sorted(fc_data, key = itemgetter('claim'))
fc_data_grouped = groupby(fc_data_sorted, key = itemgetter('claim'))

In [9]:
for key, value in groupby(fc_data_sorted, key = itemgetter('claim')):
    print(key)
    for v in value:
        print(value)
    break

'The Quiet ' is a film .


In [17]:
def map_dataset_sample(ref_entry, target_entry, score):
    entry_output = SAMPLE.copy()
    entry_output['claim'] = target_entry['claim']
    entry_output['reference'] = "{} {}".format(target_entry['claim'], ref_entry['evidence'])
    entry_output['target'] = "{} {}".format(target_entry['claim'], target_entry['evidence'])
    entry_output['score'] = score
    entry_output['label'] = target_entry['label']
    
    return entry_output


In [27]:
def key_func(k):
    return k['claim']

i = 0
ds_samples = []
fc_data_sorted = sorted(fc_data, key = key_func)

for key, values in groupby(fc_data_sorted, key_func):        
    first_value = None
    for value in values:
        if not first_value: 
            first_value = value
            continue
        if value['label'] == first_value['label']:
            continue
        # only entries with different evidence such that label flips 
        ds_samples.append(map_dataset_sample(first_value, value, score=0))

len(ds_samples)

27499

### Create examples with score == 1

In [130]:
special_chars = [
    ",",
    ".",
    "(",
    ")",
    "'",
]

In [131]:
def add_whitespace(sent: str):
    for char in special_chars: 
        sent = sent.replace(char, " {} ".format(char)).replace("  ", " ")
    print(sent+"\n")
    return sent

In [132]:
def find_in_fc_data(evidence_sent, df): 
    evid_w_whitespace = add_whitespace(evidence_sent)
    if evid_w_whitespace in list(df['evidence']):
        return df.loc[df['evidence']==evid_w_whitespace]
        
    return None

In [133]:
def map_dataset_sample_revision(fc_entry, revision_sent_a, revision_sent_b, score):
    entry_output = SAMPLE.copy()
    entry_output['claim'] = fc_entry['claim']
    entry_output['reference'] = "{} {}".format(fc_entry['claim'], revision_sent_a)
    entry_output['target'] = "{} {}".format(fc_entry['claim'], revision_sent_b)
    entry_output['score'] = score
    entry_output['label'] = fc_entry['label']
    
    return entry_output


In [134]:
# exmaples with non-factual changes 
fc_revision_non_fact = [entry for entry in fc_revision if entry['label']=='not factual']
print("{} non-factual revisions out of {}".format(len(fc_revision_non_fact), len(fc_revision)))

ds_samples = []
for entry in fc_revision_non_fact[:10]:
    # find sentence a or b in fc_data
    match = find_in_fc_data(entry['sent_a'], df_fc_data)
    if not match:
        match = find_in_fc_data(entry['sent_b'], df_fc_data)
            
    if match:
        # create example
        ds_samples.append(map_dataset_sample_revision(match, score=1))        
#         ds_samples.append(map_dataset_sample_revision(match, entry['sent_a'], entry['sent_b'], score=1))

19730 non-factual revisions out of 31048
Hickman was reunited with Mickey Rooney with whom he had acted alongside in the Boys Town sequel Men of Boys Town in 1941 . Hickman ' s book about acting techniques The Unconscious Actor: Out of Control , In Full Command] , was published in April 2007 . 

Hickman was reunited with guest-star Mickey Rooney with whom he had acted alongside in the Boys Town sequel Men of Boys Town in 1941 . Hickman ' s book about acting techniques The Unconscious Actor: Out of Control , In Full Command] , was published in April 2007 . 

Casiraghi completed two internships , firstly with the publishing house of Pierre Laffont in Paris , and then later in October 2007 , with the Sunday magazine supplement of The Independent newspaper of London . 

Casiraghi completed two internships , first with the publishing house of Pierre Laffont in Paris and then in October 2007 with the Sunday magazine supplement of The Independent newspaper of London . 

On Metacritic the film

In [135]:
len(ds_samples)

0

In [114]:
print("The channel , whose name is a concatenation of MKB ( Brownlee ' s initials ) and HD ( for high - definition ) , has over 8 . 1 million subscribers and over 1 billion total video views .")

The channel , whose name is a concatenation of MKB ( Brownlee ' s initials ) and HD ( for high - definition ) , has over 8 . 1 million subscribers and over 1 billion total video views .


In [115]:
print("The channel , whose name is a concatenation of MKB ( Brownlee 's initials ) and HD ( for high-definition ) , has over 8.57 million subscribers and over 1.29 billion total video views .")

The channel , whose name is a concatenation of MKB ( Brownlee 's initials ) and HD ( for high-definition ) , has over 8.57 million subscribers and over 1.29 billion total video views .


In [91]:
test = "test, test. (test)"

In [92]:
for char in string.punctuation: 
    test = test.replace(char, " {} ".format(char)).replace("  ", " ")

In [93]:
test

'test , test . ( test ) '